# Data pre-processing
 
**Autori:** Erik Matovič a Simon Kokavec   
**Číslo datasetu:** 165  
**Percentuálny podiel:** 50/50

## 0. Importovanie

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as py
import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms
import math

## 1. Integrácia a čistenie dát

### Zadanie  
Transformujte dáta na vhodný formát pre strojové učenie t.j. jedno pozorovanie musí byť opísané jedným riadkom a každý atribút musí byť v numerickom formáte. 
- Pri riešení chýbajúcich hodnôt (missing values) vyskúšajte rôzne stratégie z nasledujúcich podskupín:
    - odstránenie pozorovaní s chýbajúcimi údajmi
    - nahradenie chýbajúcej hodnoty mediánom, priemerom, pomerom (ku korelovanému atribútu), alebo pomocou lineárnej regresie resp. kNN
- Podobne postupujte aj pri riešení vychýlených hodnôt (outlier detection):
    - odstránenie vychýlených (odľahlých) pozorovaní
    - nahradenie vychýlenej hodnoty hraničnými hodnotami rozdelenia (5% resp. 95%)
    
### Riešenie


In [21]:
# read labor and profiles dataset
df_labor = pd.read_csv('../../data/labor.csv', sep='\t')
df_profiles = pd.read_csv('../../data/profiles.csv', sep='\t')

#### Dataset profilov
**Atribút unnamed(0)** 
- Celý stĺpec vymazať

In [105]:
df_profiles=df_profiles.drop("Unnamed: 0", axis=1)
df_profiles.head()

,name,sex,current_location,blood_group,residence,birthdate,race,ssn,address
0,Elizabeth Johnson,F,"(Decimal('-64.545933'), Decimal('5.817377'))",B-,"319 Bradley Tunnel Suite 246\nBoyerstad, GA 12067","10/30/1938, 00:00:00",White,518-82-6493,"09441 Joyce Route Suite 717\nWest Megan, GA 12969"
1,Scott Holmes,M,"(Decimal('-18.8183775'), Decimal('94.375497'))",A-,"28739 Hoffman Valleys Apt. 636\nMichaelville, ...",1927-06-20,Black,015-03-8632,USNS Miller\nFPO AE 08377
2,William Roth,M,"(Decimal('35.903361'), Decimal('-133.029344'))",B-,"8623 Jordan Canyon Suite 498\nJoshuastad, OH 3...",1965-10-02,Black,898-78-8500,"7747 Nicole Wells\nShannonside, MI 84296"
3,Kimberly Barnes,F,"(Decimal('55.2055095'), Decimal('-121.665483'))",B+,Unit 0522 Box 6156\nDPO AE 24368,1962-02-14,White,478-72-0155,"228 Colleen Extension\nLake Morgan, AL 34806"
4,Sarah Escobar,F,"(Decimal('49.512264'), Decimal('-121.259776'))",AB-,"30306 Molina Junction\nPort Edwardview, OK 72746",02 Oct 1936,Indian,683-31-6335,"08249 Gentry Fall\nLewisshire, NY 20037"


**Atribút name** 
- tento stĺpec je celý v poriadku

In [62]:
df_profiles["name"].head()

0    Elizabeth Johnson
1         Scott Holmes
2         William Roth
3      Kimberly Barnes
4        Sarah Escobar
Name: name, dtype: object

**Atribút sex** 
- Neobsahuje žiadne chýbajúce hodnoty
- tento stĺpec treba pretransformovať na numerický 
    - *1 - žena, 0 - muž*

In [63]:
df_profiles["sex"].replace({"M": 0, "F": 1}, inplace=True)

In [64]:
df_profiles = df_profiles.astype({'sex': 'bool'})

In [65]:
df_profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3064 entries, 0 to 3063
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3064 non-null   int64 
 1   name              3064 non-null   object
 2   sex               3064 non-null   bool  
 3   current_location  3064 non-null   object
 4   blood_group       3064 non-null   object
 5   residence         3064 non-null   object
 6   birthdate         3064 non-null   object
 7   race              3064 non-null   object
 8   ssn               3064 non-null   object
 9   address           3064 non-null   object
dtypes: bool(1), int64(1), object(8)
memory usage: 218.6+ KB


In [66]:
df_profiles["sex"].head()

0     True
1    False
2    False
3     True
4     True
Name: sex, dtype: bool

Tento stĺpec sme úspešne transformovali na dátový typ bool a nahradili sme jeho východiskové hodnoty M a F za False a True.

**Atribbút current_location** 
- Neobsahuje žiadne chýbajúce hodnoty


**Atribút blood_group** 
- Neobsahuje žiadne chýbajúce hodnoty
- Kategorizovať hodnoty nasledovne: 
    - **1 - A+, 2 - A-, 3 - B+, 4 - B-, 5 - AB+, 6 - AB-, 7 - O+, 8 - O-**

In [76]:
df_profiles['blood_group'] = df_profiles['blood_group'].replace({"A+": 1,"A-": 2, "B+": 3, "B-": 4, "AB+": 5, "AB-": 6, "O+": 7, "O-": 8})
df_profiles = df_profiles.astype({'blood_group': 'category'})
df_profiles.blood_group.value_counts()

1    398
3    395
8    394
7    389
5    388
2    377
6    364
4    359
Name: blood_group, dtype: int64

In [77]:
df_profiles["blood_group"].head()

0    4
1    2
2    4
3    3
4    6
Name: blood_group, dtype: category
Categories (8, int64): [1, 2, 3, 4, 5, 6, 7, 8]

**Atribút residence** 
- Neobsahuje žiadne chýbajúce hodnoty

**Atribút birthdate** 
- Neobsahuje žiadne chýbajúce hodnoty

**Atribút race** 
- Neobsahuje žiadne chýbajúce hodnoty
- Kategorizovať hodnoty nasledovne: 
    - **1 - White, 2 - Black, 3 - Asian, 4 - Hawaiian, 5 - Indian**

In [67]:
df_profiles.race.value_counts()

White       1565
Black        722
Asian        281
Hawaiian     170
Indian       159
black        102
white         36
blsck         29
Name: race, dtype: int64

Najskôr musíme upraviť hodnoty kde je chyba a ktoré majú na začiatku malé písmeno namiesto veľkého.

In [68]:
df_profiles['race'] = df_profiles['race'].replace({"blsck": "Black", "black": "Black", "white": "White"})

In [69]:
df_profiles.race.value_counts()

White       1601
Black        853
Asian        281
Hawaiian     170
Indian       159
Name: race, dtype: int64

Následne tieto hodnoty kategorizujeme ako sme uviedli vyššie.

In [70]:
df_profiles['race'] = df_profiles['race'].replace({"White": 1,"Black": 2, "Asian": 3, "Hawaiian": 4, "Indian": 5})
df_profiles = df_profiles.astype({'race': 'category'})
df_profiles.race.value_counts()

1    1601
2     853
3     281
4     170
5     159
Name: race, dtype: int64

In [72]:
df_profiles["race"].head()

0    1
1    2
2    2
3    1
4    5
Name: race, dtype: category
Categories (5, int64): [1, 2, 3, 4, 5]

**Atribút ssn** 
- Neobsahuje žiadne chýbajúce hodnoty

**Atribút address** 
- Neobsahuje žiadne chýbajúce hodnoty

#### Dataset profilov
**Atribút unnamed(0)** 
- Celý stĺpec vymazať

In [106]:
df_labor=df_labor.drop("Unnamed: 0", axis=1)
df_labor.head()

,hemoglobin,leukocyty,etytr,weight,trombocyty,ast,alp,relationship,name,alt,hbver,smoker,erytrocyty,er-cv,indicator,ssn,hematokrit
0,6.77299,5.28153,4.79087,104.22450,7.22472,61.85014,80.77401,married,Kathryn Wagner,2.01576,9.04461,no,7.94417,61.31451,0.0,887-91-9698,5.40103
1,6.35217,5.45245,3.34754,65.03608,7.00407,66.90722,90.01585,widowed,Frank Becker,0.81519,9.09262,no,5.33302,58.00624,0.0,506-37-0250,7.68167
2,5.37162,7.36106,7.10989,63.61518,6.95232,65.33091,74.23784,married,Stephanie Lewis,0.79787,6.18215,no,6.38771,62.73102,1.0,331-52-1241,6.27422
3,4.60987,8.04010,4.97779,30.35294,6.63243,67.33708,34.67995,married,Luis Williams,0.83917,5.22920,no,5.05640,62.60592,1.0,407-22-7513,6.61263
4,5.97408,6.66899,5.58872,97.97181,6.18242,67.48751,89.31030,divoced,Donald Hall,1.20122,4.97805,yes,5.10205,59.32470,1.0,486-30-7935,6.23117


**Atribút hemoglobin** 
- Obsahuje chýbajúce hodnoty

**Atribút leukocyty** 
- Obsahuje chýbajúce hodnoty

In [34]:
df_labor["leukocyty"].loc[df_labor["leukocyty"].isna(),].replace

187     NaN
390     NaN
398     NaN
563     NaN
588     NaN
640     NaN
923     NaN
1577    NaN
1950    NaN
2467    NaN
2809    NaN
2817    NaN
3368    NaN
3618    NaN
3677    NaN
4473    NaN
4658    NaN
4882    NaN
4919    NaN
5743    NaN
7035    NaN
7083    NaN
7218    NaN
7324    NaN
7555    NaN
9219    NaN
9593    NaN
9691    NaN
9869    NaN
10021   NaN
Name: leukocyty, dtype: float64

**Atribút etytr** 
- Obsahuje chýbajúce hodnoty

**Atribút weight** 
- Neobsahuje žiadne chýbajúce hodnoty
- Obsahuje mínusové hodnoty

In [27]:
weight_mean=df_labor["weight"].mean()
df_labor.loc[df_labor["weight"] < 0] = weight_mean

In [26]:
df_labor["weight"].loc[df_labor["weight"] < 0].count()

0

**Atribút trombocyty** 
- Obsahuje chýbajúce hodnoty

**Atribút ast** 
- Obsahuje chýbajúce hodnoty

**Atribút alp** 
- Obsahuje chýbajúce hodnoty

**Atribút relationship** 
- Neobsahuje žiadne chýbajúce hodnoty
- Kategorizovať hodnoty nasledovane:
    - **1 - Married, 2 - Divorced, 3 - Single, 4 - Separated, 5 - nop, 6 - widowed**

In [87]:
df_labor.relationship.value_counts()

married      2498
divoced      2488
single       2011
separated    1070
nop          1001
widowed       967
Name: relationship, dtype: int64

In [88]:
df_labor['relationship'] = df_labor['relationship'].replace({"married": 1, "divoced": 2, "single": 3, "separated": 4, "nop": 5,"widowed": 6})
df_labor = df_labor.astype({'relationship': 'category'})
df_labor.relationship.value_counts()

1    2498
2    2488
3    2011
4    1070
5    1001
6     967
Name: relationship, dtype: int64

In [89]:
df_labor["relationship"].head()

0    1
1    6
2    1
3    1
4    2
Name: relationship, dtype: category
Categories (6, int64): [1, 2, 3, 4, 5, 6]

**Atribút name** 
- Neobsahuje žiadne chýbajúce hodnoty

**Atribút alt** 
- Obsahuje chýbajúce hodnoty

**Atribút hbver** 
- Obsahuje chýbajúce hodnoty

**Atribút smoker** 
- Neobsahuje žiadne chýbajúce hodnoty
- Kategorizovať hodnoty nasledovane:
    - **1 (True) - Yes, 0 (False) - No**

In [90]:
df_labor.smoker.value_counts()

no     5441
Y      2081
yes    1965
N       548
Name: smoker, dtype: int64

In [91]:
df_labor['smoker'] = df_labor['smoker'].replace({"no": 0, "N": 0, "yes": 1, "Y": 1})
df_labor = df_labor.astype({'smoker': 'bool'})
df_labor.smoker.value_counts()

False    5989
True     4046
Name: smoker, dtype: int64

In [92]:
df_labor["smoker"].head()

0    False
1    False
2    False
3    False
4     True
Name: smoker, dtype: bool

**Atribút erytrocyty** 
- Obsahuje chýbajúce hodnoty

**Atribút er-cv** 
- Obsahuje chýbajúce hodnoty

**Atribút indicator** 
- Neobsahuje žiadne chýbajúce hodnoty
- Zmeniť dátový typ stĺpcu

In [5]:
df_labor = df_labor.astype({'indicator': 'bool'})
df_labor.indicator.value_counts()

True     6437
False    3598
Name: indicator, dtype: int64

In [6]:
df_labor["indicator"].head()

0    False
1    False
2     True
3     True
4     True
Name: indicator, dtype: bool

**Atribút ssn** 
- Neobsahuje žiadne chýbajúce hodnoty

**Atribút hematokrit** 
- Obsahuje chýbajúce hodnoty

## 2. Realizácia predspracovania dát 

### Zadanie  
- Transformované dáta pre strojové učenie si rozdeľuje na trénovaciu a testovaciu množinu (train and test dataset) podľa vami preddefinovaným pomerom. Naďalej pracujte len s trénovacím datasetom.
- Transformujte atribútov dát pre strojové učenie podľa dostupných techník (minimálne 2 techniky) ako scaling, transformers a ďalšie.
- Zdôvodnite Vašu voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie)
    
### Riešenie



## 3. Výber atribútov pre strojové učenie

### Zadanie  
- Zistite ktoré atribúty (features) vo vašich dátach pre strojové učenie sú informatívne k atribútu “indikator”.
- Zoradíte tie atribúty v poradí podľa dôležitosti. 
- Zdôvodnite Vašu voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie) 

### Riešenie




## 4. Replikovateľnosť predspracovania

### Zadanie  
- Upravte váš kód realizujúci predspracovanie trénovacej množiny tak, aby ho bolo možné bez ďalších úprav znovu použiť na predspracovanie testovacej množiny (napr. pomocou funkcie/í)
- Očakáva sa aj využitie možnosti sklearn.pipeline

### Riešenie



